In [ ]:
%matplotlib inline
import swat as sw

from dlpy.splitting import two_way_split
from dlpy.utils import create_segmentation_table
from dlpy.utils import display_segmentation_images
from dlpy.applications import UNet
from dlpy.model import AdamSolver
from dlpy.model import Optimizer
from dlpy.lr_scheduler import StepLR

In [ ]:
s = sw.CAS('dach-viya-smp.sas.com', 5570)

In [ ]:
tbl = create_segmentation_table(s,
                                path_to_images='/home/sasdemo01/airbus_ship_detection/train/',
                                path_to_ground_truth='/home/sasdemo01/airbus_ship_detection/masks/')

In [ ]:
train, validate = two_way_split(tbl, stratify=False, columns=['_filename_0', 'labels'])

In [ ]:
model = UNet(s, 
             n_classes=2, 
             n_channels=3,
             width=768,
             height=768,
             bn_after_convolutions=True)

In [ ]:
solver = AdamSolver(lr_scheduler=StepLR(learning_rate=0.0002, step_size=30, gamma=0.8), 
                    clip_grad_max = 100, 
                    clip_grad_min = -100)

optimizer = Optimizer(algorithm=solver, 
                      mini_batch_size=1, 
                      log_level=2, 
                      max_epochs=5, 
                      reg_l2=0.0005, 
                      seed=13309)

dataspecs=[dict(type='image', layer='data', data=['_image_']),
           dict(type='image', layer='Segmentation_1', data=['labels'])]

In [ ]:
model.fit(data=train,
          valid_table=validate,
          optimizer=optimizer, 
          data_specs=dataspecs, 
          n_threads=1, 
          record_seed=54321,
          force_equal_padding=True,
          gpu=dict(devices=[0]))